# VoxelGrid

In this notebook we are going to learn about how to construct and visualize a grid of voxels from a point cloud.

## Imports

In [1]:
import numpy as np

In [2]:
from pyntcloud import PyntCloud

## Setup

We are going to **load** a 3D model of an ankylosaurus from the provided `examples/data.`

You could load other point cloud in any of the supported [point cloud formats](http://pyntcloud.readthedocs.io/en/latest/io.html).

You can learn more about reading and writing point clouds in the `examples/[io]` notebooks.

In [3]:
anky = PyntCloud.from_file("data/ankylosaurus_mesh.ply")
anky

PyntCloud
34820 points with 6 scalar fields
69636 faces in mesh
0 kdtrees
0 voxelgrids
Centroid: 0.029972486197948456, 1.1250594854354858, 2.887650489807129
Other attributes:

From the description, we can see that this is a 3D mesh with 69636 faces. 

We can visualize the mesh as follows:

In [ ]:
anky.plot(mesh=True)

![mesh](data/images/structures-voxelgrid-1.png)

And convert it to a point cloud by sampling 100.000 random points from the surface.

You cand learn more about hoy to conver a triangular mesh into a point cloud in the `examples/[sampling]` notebooks.

In [4]:
anky_cloud = anky.get_sample("mesh_random", n=200000, rgb=True, normals=True, as_PyntCloud=True)

We can visualize the new point cloud:

In [ ]:
anky_cloud.plot()

![anky_cloud](data/images/structures-voxelgrid-2.png)

## VoxelGrid creation

A `VoxelGrid` is a structure generated by subdividing the minimum bounding box of a point cloud into **voxels**.

![voxelgrid](data/images/structures-voxelgrid-3.png)

You can create a `VoxelGrid` from a `PyntCloud` using the `add_structure` with the `voxelgrid` identifier and some other optional arguments.

By default, a 1x1x1 `VoxelGrid` will be created, which is not very usefull.

`pyntcloud` supports different methods for specifying how a `VoxelGrid` shold be created.

You can indicate the number of voxels per axis that you want, the desired size per axis of each voxel or a combination of both.

There are some examples bellow:

### Regular VoxelGrid using number of voxels per axis

In [5]:
voxelgrid_id = anky_cloud.add_structure("voxelgrid", n_x=64, n_y=64, n_z=64)

voxelgrid = anky_cloud.structures[voxelgrid_id]

In [ ]:
voxelgrid.plot(d=3, mode="density", cmap="hsv")

![64x64x64](data/images/structures-voxelgrid-4.png)

### Irregular VoxelGrid using number of voxels per axis

In [6]:
voxelgrid_id = anky_cloud.add_structure("voxelgrid", n_x=8, n_y=16, n_z=32)

voxelgrid = anky_cloud.structures[voxelgrid_id]

In [ ]:
voxelgrid.plot(d=3, mode="density", cmap="hsv")

![8x16x32](data/images/structures-voxelgrid-5.png)

### Regular VoxelGrid using sizes

In [7]:
voxelgrid_id = anky_cloud.add_structure("voxelgrid", size_x=0.15, size_y=0.15, size_z=0.15)

voxelgrid = anky_cloud.structures[voxelgrid_id]

In [ ]:
voxelgrid.plot(d=3, mode="density", cmap="hsv")

![0.15x0.15x0.15](data/images/structures-voxelgrid-6.png)

## Irregular VoxelGrid using sizes

In [8]:
voxelgrid_id = anky_cloud.add_structure("voxelgrid", size_x=0.01, size_y=0.05, size_z=0.15)

voxelgrid = anky_cloud.structures[voxelgrid_id]

In [ ]:
voxelgrid.plot(d=3, mode="density", cmap="hsv")

![0.01x0.05x0.15](data/images/structures-voxelgrid-7.png)

### VoxelGrid mixing number of voxels and sizes

Note that, for the same axis, **sizes** override the value of the number of voxels.

In [9]:
voxelgrid_id = anky_cloud.add_structure(
    "voxelgrid",
    n_x=32, 
    size_y=0.05, 
    # n_z gets overrided by size_z and it'll be ignored
    n_z=2, 
    size_z=0.01)

voxelgrid = anky_cloud.structures[voxelgrid_id]

In [ ]:
voxelgrid.plot(d=3, mode="density", cmap="hsv")

![32x0.05x0.01](data/images/structures-voxelgrid-8.png)

### VoxelGrid with irregular bounding box

By default, the bounding box of the voxelgrid will be adapted to make all sides of equal length. This option can be disabled as follows:

In [10]:
voxelgrid_id = anky_cloud.add_structure("voxelgrid", n_x=64, n_y=64, n_z=64, regular_bounding_box=False)

voxelgrid = anky_cloud.structures[voxelgrid_id]

In [ ]:
voxelgrid.plot(d=3, mode="density", cmap="hsv")

![irregular bounding box](data/images/structures-voxelgrid-9.png)

Note that the now the shape of the voxels is not equal for all axis, in constrast with the same construction without the regular_bounding_box argument.

In [11]:
voxelgrid.shape

[0.015778738539665937, 0.0053528593853116035, 0.009432073682546616]

## VoxelGrid Information

Once you have constructed a `VoxelGrid`, you can access many information about the structure.

Bellow is the code to get the actual VoxelGrid instance once you have added it to a PyntCloud.

In [12]:
voxelgrid_id = anky_cloud.add_structure("voxelgrid", n_x=64, n_y=64, n_z=64)

voxelgrid = anky_cloud.structures[voxelgrid_id]

### Corresponding voxel per axis for each point in the point cloud

In [13]:
voxelgrid.voxel_x

array([ 2, 31, 36, ..., 23, 24, 44], dtype=int64)

In [14]:
voxelgrid.voxel_y

array([33, 35, 39, ..., 40, 36, 31], dtype=int64)

In [15]:
voxelgrid.voxel_z

array([32, 21, 24, ..., 21, 21, 38], dtype=int64)

### Unique voxel number for each point in the point cloud

In [16]:
voxelgrid.voxel_n

array([ 10336, 129237, 149976, ...,  96789, 100629, 182246], dtype=int64)

### Number of voxels per axis

This might be usefull in case you use size arguments for constructing the structure

In [17]:
voxelgrid.x_y_z

[64, 64, 64]

### Total number of voxels

In [18]:
voxelgrid.n_voxels

262144

### Shape of the voxel

In [19]:
voxelgrid.shape

[0.015778738539665937, 0.015778739005327225, 0.015778739005327225]

### Minimum and maximum coordinates of each voxel per axis

In [20]:
min_max_x, min_max_y, min_max_z = voxelgrid.segments

In [21]:
min_max_x

array([-0.43069282, -0.41491408, -0.39913534, -0.38335661, -0.36757787,
       -0.35179913, -0.33602039, -0.32024165, -0.30446291, -0.28868417,
       -0.27290544, -0.2571267 , -0.24134796, -0.22556922, -0.20979048,
       -0.19401174, -0.17823301, -0.16245427, -0.14667553, -0.13089679,
       -0.11511805, -0.09933931, -0.08356057, -0.06778184, -0.0520031 ,
       -0.03622436, -0.02044562, -0.00466688,  0.01111186,  0.0268906 ,
        0.04266933,  0.05844807,  0.07422681,  0.09000555,  0.10578429,
        0.12156303,  0.13734177,  0.1531205 ,  0.16889924,  0.18467798,
        0.20045672,  0.21623546,  0.2320142 ,  0.24779294,  0.26357167,
        0.27935041,  0.29512915,  0.31090789,  0.32668663,  0.34246537,
        0.35824411,  0.37402284,  0.38980158,  0.40558032,  0.42135906,
        0.4371378 ,  0.45291654,  0.46869528,  0.48447401,  0.50025275,
        0.51603149,  0.53181023,  0.54758897,  0.56336771,  0.57914644])

### Voxel Centers

In [22]:
voxelgrid.voxel_centers

array([[-0.42280346,  0.62230957,  2.4119089 ],
       [-0.42280346,  0.62230957,  2.4276876 ],
       [-0.42280346,  0.62230957,  2.4434664 ],
       ...,
       [ 0.57125705,  1.6163701 ,  3.374412  ],
       [ 0.57125705,  1.6163701 ,  3.3901908 ],
       [ 0.57125705,  1.6163701 ,  3.4059696 ]], dtype=float32)

## Methods

### VoxelGrid.query

You can `query` the voxelgrid with an array of points in order to find out in which number of voxel those points
would lie.

In [23]:
some_points = np.array(
    [[0.5, 0.5, 0.5],
     [0.1, 0.1, 2.3],
     [0.49, 0.49, 0.49]])

In [24]:
voxelgrid.query(some_points)

array([237568, 135168, 237568], dtype=int64)

## VoxelGrid.get_feature_vector

You can get the 3D array representing the VoxelGrid using different modes.

#### binary

The `binary` mode will return a 3D array with 0s on empty voxels and 1s on voxels that have one point or more.

In [25]:
binary_feature_vector = voxelgrid.get_feature_vector(mode="binary")

In [26]:
binary_feature_vector.shape

(64, 64, 64)

You can count the number of occupied voxels as follow:

In [27]:
binary_feature_vector.sum()

4420.0

Which should match the number of unique voxel identifiers:

In [28]:
len(np.unique(voxelgrid.voxel_n))

4420

#### density

The `density` mode will return a 3D array with a value normalized between 0 and 1 representing the percentage of points that each voxel contains.

In [29]:
density_feature_vector = voxelgrid.get_feature_vector(mode="density")

In [30]:
density_feature_vector.shape

(64, 64, 64)

The density feature vector should sum to 1.

In [31]:
density_feature_vector.sum()

1.0

#### x_max, y_max, z_max

The `x_max`, `y_max`, `z_max` modes will return a 1D array indicating the maximum coordinate value inside a voxel, for the corresponding axis.

In [32]:
x_max_feature_vector = voxelgrid.get_feature_vector(mode="x_max")

The shape of the feature vector should match the total number of voxels

In [33]:
x_max_feature_vector.shape

(262144,)

In [34]:
voxelgrid.n_voxels

262144

The maximum and minimum `x` value of all voxels should match the maximum and minimum `x` value of the point cloud.

In [35]:
x_max_feature_vector.max()

0.5791464447975159

In [36]:
max(anky_cloud.points["x"])

0.5791464447975159